# Model 
### CALIBRATION

In [50]:
from tqdm import tqdm 
import pandas as pd 


In [14]:
# tau_ncsb, lambda_l, z_ub
beta_f   = 0.480
beta_i   = 0.525
tau_l    = 0.603 # Anton uses 0.603
tau_pi   = 1/3
tau_ncsb = 0.9   # Anton uses 0.016
lambda_l = 0.260
sigma_l  = 1.500
S        = 0.500 # Leal : 4.25
gamma    = 2/3   # Anton uses 0.33 for capital share
z_lb     = 1.000 # Leal: 1
z_ub     = 5.640 # Leal: 13
kai      = 0.4
all_vec = [0,0,0,0,0,0]
n_steps = 4

In [15]:
def labor_model_self_emp(x):
    F = np.zeros(10)

    #household eq. for eta
    F[0]  = x[0]*(1+beta_f*tau_l)-x[1]-beta_i*tau_ncsb
    #self-employment at z1
    F[1]  = x[6]-((gamma/x[1])**(1/(1-gamma))*x[2])
    #equilibrium for z1
    F[2]  = (x[1]*(1+x[6]-kai))-(x[2]**(1-gamma)*(x[6]**gamma))
    #total labor demand from employer
    F[3]  = x[3]+((1+tau_l)*(x[0]/x[1])-1)/(2*lambda_l*sigma_l*tau_l*x[5])-((gamma/((1+tau_l)*x[0]))**(1/(1-gamma))*x[5])
    #self-employment at z2
    F[4]  = x[7]-(gamma/x[1])**(1/(1-gamma))*x[5]
    #labor market equilibrium
    F[5]  = ((S/(1-S))*z_lb**S*(kai*(gamma/x[1])**(1/(1-gamma))*(x[5]**(1-S)-x[2]**(1-S))+(gamma/(1+tau_l)*x[0])**(1/(1-gamma))*(z_ub**(1-S)-x[5]**(1-S))))-1+kai*(z_lb/x[5])**S+(1-kai)*(z_lb/x[2])**S
    #goods market equilibrium
    F[6]  = tau_ncsb*((1-(z_lb/z_ub)**S)/z_lb)+x[1]*(z_lb**(-S)+kai*x[5]**(-S))-(1-kai)*x[1]*(x[2]**(-S))-(S/(1+S))*((((1+tau_l)*x[0])**2)-x[1]**2)/(2*lambda_l*sigma_l*tau_l*x[1])*(z_ub**(-S-1)-x[1]**(-S-1))*gamma**(1/(1-gamma))-(S/(1-S))*((1/x[1])**(gamma/(1-gamma))*(x[5]**(1-S)-x[2]**(1-S))+((1/(1+tau_l)*x[0])**(gamma/(1-gamma)))*(z_ub**(1-S)-x[5]**(1-S)))
    #equilibrium for z2
    F[7]  = x[2]**(1-gamma)*x[7]**gamma+x[1]*(kai-x[2])-(1-tau_pi)*(x[5]**(1-gamma)*x[7]**gamma-(1+tau_l)*x[1]*x[3]-(1+lambda_l*x[7]*x[5]*sigma_l*tau_l)*x[7])
    #government budget before reform
    F[8]  = ((gamma**(gamma/(1-gamma)))-(gamma**(1/(1-gamma))))*(1/((1+tau_l)*x[0]))**(gamma/(1-gamma))*((z_ub**(1-S)-x[5]**(1-S))/(1-S))-(((1+tau_l)**2*x[0]**2/x[1]-2*(1+tau_l)+x[1])/4*lambda_l*sigma_l*tau_l)*((z_ub**(-S-1)-x[5]**(-S-1))/(1+S))+(tau_ncsb/(tau_pi*S))*((1-x[8])*(x[2]**(-S)-z_lb**(-S))+kai*(x[5]**(-S)-x[2]**(-S))+z_ub**(-S)-x[5]**(-S))
    #share of formality
    F[9] = x[8] - ((S/(1+S))*(z_lb**S*(x[5]**(-S-1)-z_ub**(-S-1))/(1-(z_lb/z_ub)**S)*((1+tau_l)*(x[0]/x[1])-1)/(2*lambda_l*sigma_l*tau_l))) / ((S/(1-S))*(gamma/(1+tau_l)*x[0]**(1/(1-gamma))*z_lb*(z_ub**(1-S)-x[5]**(1-S))/(1-(z_lb/z_ub)**S)))

    return F

In [41]:
def labor_model_self_emp_tr(x):
    #------------------------------------------------
    # MODEL  
    #------------------------------------------------
    #parameters
    #beta_f   = 0.480
    #beta_i   = 0.525
    #tau_l    = 0.603 # Anton uses 0.603
    #tau_pi    = tau_pi_0 + tau_pi_tr
    #tau_pi_0  = 1/4
    #tau_pi_tr = .09
    #tau_ncsb = 6.8 # Anton uses 0.016
    #lambda_l = 0.24
    #sigma_l  = 1.500
    #S        = 6.3 #Leal : 4.25
    #gamma    = 2/3 #Anton uses 0.33 for capital share
    #z_lb     = 1.000 #Leal: 
    #z_ub     = 26 #Leal: 13
    #kai      = 0.4

    F = np.zeros(11)

    #household eq. for eta
    F[0]  = x[0]+beta_f*x[9]-x[1]-beta_i*tau_ncsb
    #contributory social benefits
    F[1]  = x[9]-tau_l*x[0]+((tau_pi_tr*S)/(x[8]*(x[2]**(-S)-z_lb**(-S))))*(gamma**(gamma/(1-gamma))-gamma**(1/(1-gamma)))*((1/((1+tau_l)*x[0]))**(gamma/(1-gamma)))*((z_ub**(1-S)-x[5]**(1-S))/(1-S))-((((1+tau_l)**2)*((x[0]**2)/x[1])-2*(1+tau_l)*x[0]+x[1])/(4*lambda_l*sigma_l*tau_l))*((z_ub**(-S-1)-x[5]**(-S-1))/(1+S))
    #self-employment at z1
    F[2]  = x[6]-((gamma/x[1])**(1/(1-gamma))*x[2])
    #equilibrium for z1
    F[3]  = (x[1]*(1+x[6]-kai))-(x[2]**(1-gamma)*(x[6]**gamma))
    #total labor demand from employer
    F[4]  = x[3]+((1+tau_l)*(x[0]/x[1])-1)/(2*lambda_l*sigma_l*tau_l*x[5])-((gamma/((1+tau_l)*x[0]))**(1/(1-gamma))*x[5])
    #self-employment at z2
    F[5]  = x[7]-(gamma/x[1])**(1/(1-gamma))*x[5]
    #labor market equilibrium
    F[6]  = ((S/(1-S))*z_lb**S*(kai*(gamma/x[1])**(1/(1-gamma))*(x[5]**(1-S)-x[2]**(1-S))+(gamma/(1+tau_l)*x[0])**(1/(1-gamma))*(z_ub**(1-S)-x[5]**(1-S))))-1+kai*(z_lb/x[5])**S+(1-kai)*(z_lb/x[2])**S
    #goods market equilibrium
    F[7]  = tau_ncsb*((1-(z_lb/z_ub)**S)/z_lb)+x[1]*(z_lb**(-S)+kai*x[5]**(-S))-(1-kai)*x[1]*(x[2]**(-S))-(S/(1+S))*((((1+tau_l)*x[0])**2)-x[1]**2)/(2*lambda_l*sigma_l*tau_l*x[1])*(z_ub**(-S-1)-x[1]**(-S-1))*gamma**(1/(1-gamma))-(S/(1-S))*((1/x[1])**(gamma/(1-gamma))*(x[5]**(1-S)-x[2]**(1-S))+((1/(1+tau_l)*x[0])**(gamma/(1-gamma)))*(z_ub**(1-S)-x[5]**(1-S)))
    #equilibrium for z2
    F[8]  = x[2]**(1-gamma)*x[7]**gamma+x[1]*(kai-x[2])-(1-tau_pi_0-tau_pi_tr)*(x[5]**(1-gamma)*x[7]**gamma-(1+tau_l)*x[1]*x[3]-(1+lambda_l*x[7]*x[5]*sigma_l*tau_l)*x[7])
    #government budget before reform
    F[9]  = ((gamma**(gamma/(1-gamma)))-(gamma**(1/(1-gamma))))*(1/((1+tau_l)*x[0]))**(gamma/(1-gamma))*((z_ub**(1-S)-x[5]**(1-S))/(1-S))-(((1+tau_l)**2*x[0]**2/x[1]-2*(1+tau_l)+x[1])/4*lambda_l*sigma_l*tau_l)*((z_ub**(-S-1)-x[5]**(-S-1))/(1+S))+(tau_ncsb/((tau_pi_0+tau_pi_tr)*S))*((1-x[8])*(x[2]**(-S)-z_lb**(-S))+kai*(x[5]**(-S)-x[2]**(-S))+z_ub**(-S)-x[5]**(-S))
    #share of formality
    F[10] = x[8] - ((S/(1+S))*(z_lb**S*(x[5]**(-S-1)-z_ub**(-S-1))/(1-(z_lb/z_ub)**S)*((1+tau_l)*(x[0]/x[1])-1)/(2*lambda_l*sigma_l*tau_l))) / ((S/(1-S))*(gamma/(1+tau_l)*x[0]**(1/(1-gamma))*z_lb*(z_ub**(1-S)-x[5]**(1-S))/(1-(z_lb/z_ub)**S)))

    return F

In [42]:
import numpy as np
from scipy.optimize import least_squares

In [49]:
x0 = [1,    1,   1,  1,  1,  1,  1,  1, .5]
lb = [0,    0,z_lb,  0,  0,z_lb,  0,  0,0]
ub = [100,100,z_ub,100,100,z_ub,100,100,1]
all_vec = []

for tau_ncsb in tqdm(np.linspace(7.000,7.200,n_steps)):
    for lambda_l in np.linspace(0.45,0.55,n_steps):
        for z_ub in np.linspace(1.150,1.250,n_steps):
            for S in np.linspace(0.500,0.600,n_steps):
                res = least_squares(labor_model_self_emp,x0,bounds=(lb,ub))
                # results
                w_f_w_l = res.x[0]/res.x[1]
                shr_inf_emp = (1-res.x[8])*100 
                # results of interest
                all_vec.append([tau_ncsb,lambda_l,z_ub,S,shr_inf_emp,w_f_w_l])
all_vec = pd.DataFrame(all_vec,columns=['tau_ncsb','lambda_l','z_ub','S','shr_inf_emp','w_f_w_l'])

100%|██████████| 4/4 [00:13<00:00,  3.31s/it]


In [51]:
all_vec.describe()

,tau_ncsb,lambda_l,z_ub,S,shr_inf_emp,w_f_w_l
count,256.000000,256.000000,256.000000,256.000000,256.000000,256.000000
mean,7.100000,0.500000,1.200000,0.550000,93.625389,1.927537
std,0.074682,0.037341,0.037341,0.037341,0.860563,0.057407
min,7.000000,0.450000,1.150000,0.500000,90.919839,1.820723
25%,7.050000,0.475000,1.175000,0.525000,93.051154,1.878205
50%,7.100000,0.500000,1.200000,0.550000,93.679519,1.928287
75%,7.150000,0.525000,1.225000,0.575000,94.269266,1.976950
max,7.200000,0.550000,1.250000,0.600000,95.329132,2.031595


In [38]:
# model without tax reform
#------------------------
# parameters
beta_f   = 0.480
beta_i   = 0.525
tau_l    = 0.603 # Anton uses 0.603
tau_pi   = 1/3
tau_ncsb = 7.000 # Anton uses 0.016
lambda_l = 0.500
sigma_l  = 1.500
S        = 0.510 # Leal : 4.25
gamma    = 2/3   # Anton uses 0.33 for capital share
z_lb     = 1.000 # Leal: 1
z_ub     = 1.2  # Leal: 13
kai      = 0.4
# simulation
x0 = [1,    1,   1,  1,  1,  1,  1,  1, .5]
lb = [0,    0,z_lb,  0,  0,z_lb,  0,  0, 0]
ub = [100,100,z_ub,100,100,z_ub,100,100, 1]

res = least_squares(labor_model_self_emp,x0,bounds=(lb,ub))
# results
w_f_w_l = res.x[0]/res.x[1]
shr_inf_emp = (1-res.x[8])*100 
print('Ratio:',w_f_w_l )
print('share:',shr_inf_emp)

Ratio: 1.9392596203167445
share: 93.64539141796982


In [40]:
# model with tax reform
#---------------------
# parameters
beta_f    = 0.480
beta_i    = 0.525
tau_l     = 0.468 # Anton uses 0.603
tau_pi_0  = 1/4
tau_pi_tr = 0.09
tau_ncsb = 7.00 # Anton uses 0.016
lambda_l = 0.500
sigma_l  = 1.500
S        = 0.51 # Leal : 4.25
gamma    = 2/3 # Anton uses 0.33 for capital share
z_lb     = 1.000 # Leal: 1
z_ub     = 1.2 # Leal: 13
kai      = 0.4

# simulation
x0 = [2,    1, 1.1,  1,  1, 1.1,  1,  1,.5,  1]
lb = [0,    0,z_lb,  0,  0,z_lb,  0,  0, 0,  0]
ub = [100,100,z_ub,100,100,z_ub,100,100, 1,100]

res = least_squares(labor_model_self_emp_tr,x0,bounds=(lb,ub))

# results
w_f_w_l = res.x[0]/res.x[1]
shr_inf_emp_tr = (1-res.x[8])*100 
print('Ratio:',w_f_w_l )
print('share:',shr_inf_emp)

# effect tax reform on informality
change_shr_inf_emp     =  shr_inf_emp_tr-shr_inf_emp 
change_shr_inf_emp_per =((shr_inf_emp_tr-shr_inf_emp)/shr_inf_emp)*100
print('change share:',change_shr_inf_emp)
print('change share per:',change_shr_inf_emp_per)


Ratio: 1.855981724328794
share: 93.64539141796982
change share: -91.04922637712947
change share per: -97.2276638481302
